# Decumulate rain fields

and write it back to GRIB, ensuring proper validity

In [1]:
%matplotlib inline
# for figures in notebook

# import & initialize epygram
import epygram
epygram.init_env()

import os
INPUTS_DIR = os.path.join(epygram.config.installdir, 'doc_sphinx', 'source', 'new_gallery', 'inputs')
OUTPUTS_DIR = os.path.join(epygram.config.installdir, 'doc_sphinx', 'source', 'new_gallery', 'outputs')

In [2]:
paths = [os.path.join(INPUTS_DIR, 'grid.arome-forecast.guyane0025+00{:02}:00.grib'.format(i))
         for i in (0, 12, 24)]
resources = [epygram.formats.resource(p, 'r') for p in paths]

In [4]:
rain = [r.readfield({'discipline':0, 'parameterCategory':1, 'parameterNumber':65})
        for r in resources[1:]]  # start from 1: term 0, no rain

In [5]:
# rain is accumulated from beginning of forecast
print([r.mean() for r in rain])

[6.270696104762613, 12.545468328417483]


In [6]:
for i in range(len(rain)):
    if i == 0: continue  # term 1, no decumulation to do
    rain[i].operation('-', rain[i-1])
    rain[i].validity[0].set(cumulativeduration=(rain[i].validity[0].get()-rain[i-1].validity[0].get()))

In [7]:
print([r.mean() for r in rain])

[6.270696104762613, 6.27477222365487]


In [8]:
rain_series = epygram.formats.resource(os.path.join(OUTPUTS_DIR,'rain_series.grib'), 'w', fmt='GRIB')
for r in rain:
    print(r.validity[0])
    rain_series.writefield(r)
rain_series.close()

FieldValidity containing:
    _basis: 2020-11-02 00:00:00
    _date_time: 2020-11-02 12:00:00
    _cumulativeduration: 12:00:00
    _statistical_process_on_duration: 1
    _statistical_time_increment: 0:00:00
FieldValidity containing:
    _basis: 2020-11-02 00:00:00
    _date_time: 2020-11-03 00:00:00
    _cumulativeduration: 12:00:00
    _statistical_process_on_duration: 1
    _statistical_time_increment: 0:00:00
